In [5]:
import sys
import os
import se
import pandas as pd
import tree_util
import copy
import numpy as np

outer = 'result/GCN_fix_tree'
with open(os.path.join(outer, 'renamed_GCN_tree.newick')) as fp:
    newick_tree = fp.read()

json_tree = tree_util.parse(newick_tree)
largest = {'largest': 0}
leaf_list, l = tree_util.recu_compute(json_tree, 0, largest)
largest_level = largest['largest']
nlayer = largest_level
leaf_list, l = tree_util.recu_compute(json_tree, 0, largest)
layer_leaves_dict = tree_util.make_layer_dict(nlayer)

tree_util.recu_layer(json_tree, layer_leaves_dict)
tree_util.to_layer_leaves(layer_leaves_dict, nlayer)

result = {}
# compute leaf layer
result['leaves_dict'] = copy.deepcopy(layer_leaves_dict)
parent_dict = {}
tree_util.parents(json_tree, parent_dict)
node_leaves = {}
for level in layer_leaves_dict.keys():
    for node, sp_list in layer_leaves_dict[level].items():
        if node in node_leaves.keys():
            continue
        node_leaves[node] = copy.deepcopy(sp_list)
subtree_nodes = {}
for l in leaf_list:
    parent = parent_dict[l]
    if parent not in subtree_nodes.keys():
        subtree_nodes[parent] = []
    subtree_nodes[parent].append(l)

for node in node_leaves.keys():
    parent = parent_dict[node]
    if parent not in subtree_nodes.keys():
        subtree_nodes[parent] = []
    subtree_nodes[parent] += subtree_nodes[node]
    subtree_nodes[parent].append(node)

for node in subtree_nodes.keys():
    subtree_nodes[node].append(node)

direct_children_dict = {}
for node, parent in parent_dict.items():
    if parent not in direct_children_dict:
        direct_children_dict[parent] = []
    direct_children_dict[parent].append(node)

result['leaves_dict']['root'] = {'root': copy.deepcopy(leaf_list)}
node_leaves['root'] = copy.deepcopy(leaf_list)


In [3]:
subtree_nodes.keys()

dict_keys(['cluster_S1-C1', 'cluster_S1-C2', 'cluster_S1-C3', 'cluster_S1-C4', 'cluster_S1-C5', 'cluster_S1-C6', 'cluster_S1-C7', 'cluster_S1-C8', 'cluster_S1-C9', 'cluster_S1-C10', 'cluster_S1-C11', 'cluster_S1-C12', 'cluster_S1-C13', 'cluster_S1-C14', 'cluster_S1-C15', 'cluster_S1-C16', 'cluster_S1-C17', 'cluster_S1-C18', 'cluster_S1-C19', 'cluster_S1-C20', 'cluster_S1-C21', 'cluster_S1-C22', 'cluster_S1-C23', 'cluster_S1-C24', 'cluster_S1-C25', 'cluster_S1-C26', 'cluster_S1-C27', 'cluster_S1-C28', 'cluster_S2-C1', 'cluster_S2-C2', 'cluster_S2-C3', 'cluster_S2-C4', 'cluster_S2-C5', 'cluster_S2-C6', 'cluster_S2-C7', 'cluster_S2-C8', 'cluster_S3-C1', 'cluster_S3-C2', 'cluster_S3-C3', 'cluster_S3-C4', 'cluster_S3-C5', 'cluster_S4-C1', 'cluster_S4-C2', 'cluster_S4-C3', 'cluster_S4-C4', 'cluster_S5-C1', 'cluster_S5-C2', 'cluster_S5-C3', 'cluster_S5-C4', 'cluster_S6-C1', 'cluster_S6-C2', 'cluster_S6-C3', 'cluster_C7', 'cluster_C8', 'cluster_C9', 'cluster_C10', 'supercluster_S1', 'superclus

In [8]:
param = 0.5
edge_df = pd.DataFrame(index=leaf_list, columns=leaf_list, data=np.ones((len(leaf_list), len(leaf_list))) * param)
np.fill_diagonal(edge_df.values, 0)


In [10]:
result = se.subtree_se_adj(edge_df, parent_dict, node_leaves, subtree_nodes, direct_children_dict, param)

In [14]:
sorted(subtree_nodes['supercluster_S1'])

['cluster_S1-C1',
 'cluster_S1-C10',
 'cluster_S1-C11',
 'cluster_S1-C12',
 'cluster_S1-C13',
 'cluster_S1-C14',
 'cluster_S1-C15',
 'cluster_S1-C16',
 'cluster_S1-C17',
 'cluster_S1-C18',
 'cluster_S1-C19',
 'cluster_S1-C2',
 'cluster_S1-C20',
 'cluster_S1-C21',
 'cluster_S1-C22',
 'cluster_S1-C23',
 'cluster_S1-C24',
 'cluster_S1-C25',
 'cluster_S1-C26',
 'cluster_S1-C27',
 'cluster_S1-C28',
 'cluster_S1-C3',
 'cluster_S1-C4',
 'cluster_S1-C5',
 'cluster_S1-C6',
 'cluster_S1-C7',
 'cluster_S1-C8',
 'cluster_S1-C9',
 's--Abiotrophia-defectiva',
 's--Abiotrophia-sp-HMSC24B09',
 's--Absiella-dolichum',
 's--Acetatifactor-muris',
 's--Acetivibrio-ethanolgignens',
 's--Acetobacter-orientalis',
 's--Acetobacteraceae-bacterium-DB1506',
 's--Acetomicrobium-flavidum',
 's--Acholeplasma-oculi',
 's--Achromobacter-arsenitoxydans',
 's--Achromobacter-denitrificans',
 's--Achromobacter-insolitus',
 's--Achromobacter-insuavis',
 's--Achromobacter-marplatensis',
 's--Achromobacter-mucicolens',
 's-

In [19]:
print(result['supercluster_S1'])
total = 0
for k, v in result.items():
    if k.startswith('cluster_S1'):
        print(k, v)
        total += v
print(total)

6.8463219818321255
cluster_S1-C1 0.6130494330950832
cluster_S1-C2 0.5561991899512377
cluster_S1-C3 0.5434770798192143
cluster_S1-C4 0.4793323890237429
cluster_S1-C5 0.44689595405188365
cluster_S1-C6 0.420753111221325
cluster_S1-C7 0.41418888542285137
cluster_S1-C8 0.39442431986290855
cluster_S1-C9 0.39442431986290855
cluster_S1-C10 0.3878113904711963
cluster_S1-C11 0.3878113904711963
cluster_S1-C12 0.37454692680877144
cluster_S1-C13 0.34114765286415816
cluster_S1-C14 0.3005736561103371
cluster_S1-C15 0.2800600687177101
cluster_S1-C16 0.27318558901364454
cluster_S1-C17 0.24548994847266872
cluster_S1-C18 0.2385134897914581
cluster_S1-C19 0.23151462604857914
cluster_S1-C20 0.2103756023506642
cluster_S1-C21 0.181822789908356
cluster_S1-C22 0.181822789908356
cluster_S1-C23 0.16009178915313532
cluster_S1-C24 0.1454312096319267
cluster_S1-C25 0.1380432314391832
cluster_S1-C26 0.10804789092460095
cluster_S1-C27 0.10804789092460095
cluster_S1-C28 0.07716617265503375
8.634248787976732


In [20]:
edge_df1 = copy.deepcopy(edge_df)
v_g = edge_df1.sum().sum()
#v_g = 1
se_dict, outside_dict = se.all_node_se(edge_df1, parent_dict, node_leaves)
#print(len(outside_dict), len(outside_dict))
log_s_dict = se.all_gamma(direct_children_dict, outside_dict, node_leaves, v_g)

In [22]:
log_s_dict['supercluster_S1']

0.5266692266015327

In [23]:
node = 'supercluster_S1'
children_list = subtree_nodes[node]
value = se.se_sum(se_dict, children_list, node) + param*se.adj_sum(log_s_dict, children_list, node)


In [46]:
se_sum = 0
for c in children_list:
    se_sum += se_dict[c]

g_sum = 0
for c in children_list:
    if c in log_s_dict.keys():
        g_sum += log_s_dict[c]
se_sum + g_sum*0.5

6.392677323101763

In [49]:
total_value = 0
for c in direct_children_dict['supercluster_S1']:
    node = c
    se_sum = 0
    children_list1 = subtree_nodes[node]
    for c in children_list1:
        se_sum += se_dict[c]

    g_sum = 0
    for c in children_list1:
        if c in log_s_dict.keys():
            g_sum += log_s_dict[c]
    v = se_sum + g_sum*0.5
    print(node, v)
    total_value += v
total_value


cluster_S1-C1 0.44806604683640666
cluster_S1-C2 0.40210325013149334
cluster_S1-C3 0.39189356073468345
cluster_S1-C4 0.34086837971124156
cluster_S1-C5 0.31537033042334867
cluster_S1-C6 0.29497887077933754
cluster_S1-C7 0.28988197528298704
cluster_S1-C8 0.27459361538883154
cluster_S1-C9 0.27459361538883154
cluster_S1-C10 0.26949827095568024
cluster_S1-C11 0.26949827095568024
cluster_S1-C12 0.2593087453865935
cluster_S1-C13 0.23384171736301104
cluster_S1-C14 0.20329407999627777
cluster_S1-C15 0.18802549614378314
cluster_S1-C16 0.18293674372293864
cluster_S1-C17 0.16258561168653357
cluster_S1-C18 0.15749879808845643
cluster_S1-C19 0.1524123722544329
cluster_S1-C20 0.13715542133429962
cluster_S1-C21 0.1168182487861778
cluster_S1-C22 0.1168182487861778
cluster_S1-C23 0.10156944086584735
cluster_S1-C24 0.09140550771087412
cluster_S1-C25 0.08632412276671704
cluster_S1-C26 0.06600246048616358
cluster_S1-C27 0.06600246048616358
cluster_S1-C28 0.045687001918398557


5.93903266437137

In [50]:
total_value + se_dict['supercluster_S1'] + 0.5*log_s_dict['supercluster_S1']

6.392677323101732

In [39]:
len(children_list)

1197

In [40]:
len(children_list1)

10

In [42]:
len(node_leaves['supercluster_S1'])

1168

In [43]:
1197-1168

29

In [44]:
set(children_list).difference(set(node_leaves['supercluster_S1']))

{'cluster_S1-C1',
 'cluster_S1-C10',
 'cluster_S1-C11',
 'cluster_S1-C12',
 'cluster_S1-C13',
 'cluster_S1-C14',
 'cluster_S1-C15',
 'cluster_S1-C16',
 'cluster_S1-C17',
 'cluster_S1-C18',
 'cluster_S1-C19',
 'cluster_S1-C2',
 'cluster_S1-C20',
 'cluster_S1-C21',
 'cluster_S1-C22',
 'cluster_S1-C23',
 'cluster_S1-C24',
 'cluster_S1-C25',
 'cluster_S1-C26',
 'cluster_S1-C27',
 'cluster_S1-C28',
 'cluster_S1-C3',
 'cluster_S1-C4',
 'cluster_S1-C5',
 'cluster_S1-C6',
 'cluster_S1-C7',
 'cluster_S1-C8',
 'cluster_S1-C9',
 'supercluster_S1'}

In [45]:
node in subtree_nodes[node]

True